In [1]:
import sys,os
dev_mode = True
preset = 'python'

if dev_mode:
    try:
        install_prefix = os.environ['CMAKE_INSTALL_PREFIX']
        print(f'Using CMAKE_INSTALL_PREFIX={install_prefix}')
    except KeyError:
        install_prefix = f'../../build/{preset}/install'
print(f'Using install_prefix={install_prefix}')
if dev_mode and os.path.exists(f'{install_prefix}/lib/python3/dist-packages'):
    sys.path.insert(0, os.path.abspath(f'{install_prefix}/lib/python3/dist-packages'))
print(sys.path)

Using install_prefix=../../build/python/install
['/scratch/prudhomm/Devel/feelpp.quickfix/build/python/install/lib/python3/dist-packages', '/scratch/prudhomm/Devel/feelpp.quickfix/python/notebooks', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/u2/prudhomm/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/local/lib/python3/dist-packages/', '/usr/lib/python3/dist-packages', '/usr/lib/petsc/lib/python3/dist-packages', '/usr/lib/slepc/lib/python3/dist-packages', '/usr/lib/python3/dist-packages/IPython/extensions', '/home/u2/prudhomm/.ipython']


In [2]:
import feelpp
from feelpp.meshmover import *
from feelpp.integrate import integrate
from feelpp.measure import measure
import feelpp.toolboxes.core as core
from feelpp.toolboxes.cfpdes import *
import feelpp.quality as quality
import numpy as np
print(f"feelpp loaded from {feelpp.__file__}")

feelpp loaded from /scratch/prudhomm/Devel/feelpp.quickfix/build/python/install/lib/python3/dist-packages/feelpp/__init__.py


In [3]:
os.getcwd()

'/scratch/prudhomm/Devel/feelpp.quickfix/build/python/install/lib/python3/dist-packages'

In [4]:
sys.argv = ['notebook-mesh']
e = feelpp.Environment(sys.argv,config=feelpp.localRepository(sys.argv[0]),opts = core.toolboxes_options("coefficient-form-pdes", "cfpdes"))

In [5]:
os.getcwd()

'/scratch/prudhomm/Devel/feelpp.quickfix/build/python/install/lib/python3/dist-packages/feelppdb/notebook-mesh/np_1'

In [6]:
geo = {
    '2': feelpp.create_rectangle(),
    '3': feelpp.create_box(),
}
mesh_name = geo['2'][0]
print(mesh_name)
mesh = feelpp.load(feelpp.mesh(dim=2), geo['2'][0], 0.1)

rectangle-1.msh


In [7]:
laplacian={
    "Name": "Laplacian",
    "ShortName": "Laplacian",
    "Models":
    {
        "cfpdes":{ "equations":"laplace" },
        "laplace":{
            "setup":{
                "unknown":{
                    "basis":"Pchv1",
                    "name":"u",
                    "symbol":"u"
                },
                "coefficients":{ "c":"1", "f":"{1,1}"}
            }
        }
    },
    "Materials": { "Omega": { "markers":"Omega" } },
    "BoundaryConditions":
    {
        "laplace":
        {
            "Dirichlet":
            {
                "GammaMove":
                {
                    "markers":["Gamma_1"],
                    "expr":"{0,amplitude*x*(1-x)+0.1}:x:amplitude"
                },
                "zero":
                {   
                    "markers":["Gamma_2","Gamma_3","Gamma_4"],
                    "expr":"{0,0}"
                }
            }
        }
    },
    "PostProcess":
    {
        "cfpdes":
        {
            "Exports":
            {
                "fields":["all"]
            }
        }
    }
}


In [8]:

print("Test of the mesh quality")
hclose = 0.01
hfar = 0.05
q = quality.etaQ(mesh).min()
print(f"mesh quality criterion={q}\n")
if q < 0.4 : 
    print("Remeshing\n")
    mesh, cpt = feelpp.remesh(mesh=mesh, metric=f"gradedls({hclose},{hfar})",required_elts=[],required_facets=[],parent=None)

for iter in range(0,3):
    toolbox=cfpdes(dim=2)
    toolbox.setMesh(mesh)
    toolbox.setModelProperties(laplacian)
    toolbox.addParameterInModelProperties("amplitude",0.2)

    #
    toolbox.init(buildModelAlgebraicFactory=True)
    toolbox.solve()
    toolbox.exportResults()

    u=toolbox.pde("laplace").fieldUnknown()
    #assert(abs(M-2)<1e-10)
    M=measure(range=feelpp.elements(mesh))  
    print(f"[iter {iter}] before : {M}")
    mesh=meshMove(mesh,u)
    M=measure(range=feelpp.elements(mesh))   
    print(f"[iter {iter}] after : {M}")
    q = quality.etaQ(mesh).min()
    print(f"mesh quality criterion={q}\n")
    if q < 0.4 : 
        print("Remeshing\n")
        mesh, cpt = feelpp.remesh(mesh=mesh, metric=f"gradedls({hclose},{hfar})",required_elts=[],required_facets=[],parent=None)

Test of the mesh quality
mesh quality criterion=0.8492776442582294

[iter 0] before : 2.0
[iter 0] after : 1.9999999999999998
mesh quality criterion=0.557962765518826

[iter 1] before : 1.9999999999999998
[iter 1] after : 2.0000000000000004
mesh quality criterion=0.2280936499003176

Remeshing

[iter 2] before : 2.007513892634847
[iter 2] after : 2.008890058773225
mesh quality criterion=0.002542647691316815

Remeshing

